In [1]:
import sympy as sm
from sympy import diff, Symbol, cos,diag
from scipy.integrate import solve_ivp,OdeSolver,odeint
import numpy as np
from sympy.physics.mechanics import dynamicsymbols
import math
import matplotlib.pyplot as plt
from itertools import combinations_with_replacement
from functools import reduce

In [2]:
from sympy.physics.vector import init_vprinting
init_vprinting(use_latex="mathjax", pretty_print=False)

In [3]:
m1, m2, l1, l2, gty ,tau1, tau2,tau3,tau4, c1, c2, c3, c4, L, I1, I2 = sm.symbols('m1 m2 l1 l2 g tau1 tau2 tau3 tau4 c1 c2 c3 c4 L I1 I2', real =True)
t = Symbol('t')
theta1, theta2, phi1, phi2 =  dynamicsymbols('theta1 theta2 phi1 phi2')

theta1d = dynamicsymbols('theta1',1)
theta2d = dynamicsymbols('theta2',1)
phi1d = dynamicsymbols('phi1',1)
phi2d = dynamicsymbols('phi2',1)
theta1dd = dynamicsymbols('theta1',2)
theta2dd = dynamicsymbols('theta2',2)
phi1dd = dynamicsymbols('phi1',2)
phi2dd = dynamicsymbols('phi2',2)
gty, m1, m2, l1, l2, theta1, theta2, phi1, phi2, theta1d, theta2d, phi1d,phi2d, theta1dd, theta2dd, phi1dd,phi2dd, tau1, tau2,tau3,tau4, I1, I2

(g, m1, m2, l1, l2, theta1, theta2, phi1, phi2, theta1', theta2', phi1', phi2', theta1'', theta2'', phi1'', phi2'', tau1, tau2, tau3, tau4, I1, I2)

In [4]:
x1 = l1*sm.cos(theta1)*sm.sin(phi1)
y1 = l1*sm.sin(theta1)*sm.sin(phi1)
z1 = l1*sm.cos(phi1)

x2 = x1 + l2*sm.cos(theta2)*sm.sin(phi2)
y2 = y1 + l2*sm.sin(theta2)*sm.sin(phi2)
z2 = z1 + l2*cos(phi2 )

x1_dot = sm.diff(x1,t)
x2_dot = sm.diff(x2,t)
y1_dot = sm.diff(y1,t)
y2_dot = sm.diff(y2,t)
z1_dot = sm.diff(z1,t)
z2_dot = sm.diff(z2,t)

x1_dot , x2_dot, y1_dot , y2_dot, z1_dot, z2_dot

(-l1*sin(phi1)*sin(theta1)*theta1' + l1*cos(phi1)*cos(theta1)*phi1', -l1*sin(phi1)*sin(theta1)*theta1' + l1*cos(phi1)*cos(theta1)*phi1' - l2*sin(phi2)*sin(theta2)*theta2' + l2*cos(phi2)*cos(theta2)*phi2', l1*sin(phi1)*cos(theta1)*theta1' + l1*sin(theta1)*cos(phi1)*phi1', l1*sin(phi1)*cos(theta1)*theta1' + l1*sin(theta1)*cos(phi1)*phi1' + l2*sin(phi2)*cos(theta2)*theta2' + l2*sin(theta2)*cos(phi2)*phi2', -l1*sin(phi1)*phi1', -l1*sin(phi1)*phi1' - l2*sin(phi2)*phi2')

# dynamic modeling on manipulator without constraint using lagrange equation

In [5]:
# kinetic energy
ke = (sm.Rational(1/2))*m1*(x1_dot**2)
ke += (sm.Rational(1/2))*m1*(y1_dot**2)
ke += (sm.Rational(1/2))*m1*(z1_dot**2)
ke += (sm.Rational(1/2))*m2*(x2_dot**2)
ke += (sm.Rational(1/2))*m2*(y2_dot**2)
ke += (sm.Rational(1/2))*m2*(z2_dot**2)
sm.simplify(ke)

l1**2*m1*sin(phi1)**2*theta1'**2/2 + l1**2*m1*phi1'**2/2 + l1**2*m2*sin(phi1)**2*theta1'**2/2 + l1**2*m2*phi1'**2/2 - l1*l2*m2*sin(theta1 - theta2)*sin(phi1)*cos(phi2)*phi2'*theta1' + l1*l2*m2*sin(theta1 - theta2)*sin(phi2)*cos(phi1)*phi1'*theta2' + l1*l2*m2*sin(phi1)*sin(phi2)*cos(theta1 - theta2)*theta1'*theta2' + l1*l2*m2*sin(phi1)*sin(phi2)*phi1'*phi2' + l1*l2*m2*cos(theta1 - theta2)*cos(phi1)*cos(phi2)*phi1'*phi2' + l2**2*m2*sin(phi2)**2*theta2'**2/2 + l2**2*m2*phi2'**2/2

In [6]:
# potential energy
pe = m1*gty*y1
pe += m2*gty*y2
sm.expand(pe)

g*l1*m1*sin(phi1)*sin(theta1) + g*l1*m2*sin(phi1)*sin(theta1) + g*l2*m2*sin(phi2)*sin(theta2)

In [7]:
# the lagrangian equation
Ls = ke - pe

In [8]:
f1 = (sm.diff(sm.diff(Ls,theta1d),t) - sm.diff(Ls,theta1)).expand()
f2 = (sm.diff(sm.diff(Ls,theta1d),t) - sm.diff(Ls,theta2)).expand()
f3 = (sm.diff(sm.diff(Ls,phi1d),t) - sm.diff(Ls,phi1)).expand()
f4 = (sm.diff(sm.diff(Ls,phi2d),t) - sm.diff(Ls,phi2)).expand()

In [9]:
#inertia term
rows = []
for f in [f1,f2,f3,f4]:
    columns = []
    for v in [theta1dd,theta2dd,phi1dd,phi2dd]:
        columns.append(f.coeff(v))
    rows.append(columns)

M = sm.Matrix(rows)
M

Matrix([
[l1**2*m1*sin(phi1)**2*sin(theta1)**2 + l1**2*m1*sin(phi1)**2*cos(theta1)**2 + l1**2*m2*sin(phi1)**2*sin(theta1)**2 + l1**2*m2*sin(phi1)**2*cos(theta1)**2, l1*l2*m2*sin(phi1)*sin(phi2)*sin(theta1)*sin(theta2) + l1*l2*m2*sin(phi1)*sin(phi2)*cos(theta1)*cos(theta2),                                                                                                                                                                                                         0,                               -l1*l2*m2*sin(phi1)*sin(theta1)*cos(phi2)*cos(theta2) + l1*l2*m2*sin(phi1)*sin(theta2)*cos(phi2)*cos(theta1)],
[l1**2*m1*sin(phi1)**2*sin(theta1)**2 + l1**2*m1*sin(phi1)**2*cos(theta1)**2 + l1**2*m2*sin(phi1)**2*sin(theta1)**2 + l1**2*m2*sin(phi1)**2*cos(theta1)**2, l1*l2*m2*sin(phi1)*sin(phi2)*sin(theta1)*sin(theta2) + l1*l2*m2*sin(phi1)*sin(phi2)*cos(theta1)*cos(theta2),                                                                                                                      

In [10]:
#centrifugal / Coriolis force term,
rows = []
coordinates = list(combinations_with_replacement([theta1d,theta2d,phi1d,phi2d], 2))
c_v = [sm.prod(i) for i in coordinates]
for f in [f1,f2,f3,f4]:
    columns = []
    for v in c_v:
        columns.append(f.coeff(v)*v)
    rows.append(sum(columns))

C = sm.Matrix(rows)

In [11]:
# gravity term
rows = []
for f in [f1,f2,f3,f4]:
    columns = []
    for v in [theta1dd,theta2dd,phi1dd,phi2dd]:
        columns.append(f.coeff(v)*v)
    rows.append(sum(columns))

G = sm.Matrix([f1,f2,f3,f4]) - sm.Matrix(rows) - C

In [12]:
tau = sm.Matrix([tau1, tau2, tau3, tau4])
tau

Matrix([
[tau1],
[tau2],
[tau3],
[tau4]])

In [13]:
# added friction
d = diag(c1, c2, c3, c4) * sm.Matrix([theta1d, theta2d, phi1d, phi2d])
F = tau - C - G + d
F.shape

(4, 1)

# Dynamic modeling subject to constraint

In [14]:
endefX = x2 - L #endeffector x trajectory
endefY = y2 - 0.2*(1-sm.cos(t)) #endeffector y trajectory
endefZ = z2 - L #endeffector z trajectory
endefX, endefY, endefZ

(-L + l1*sin(phi1)*cos(theta1) + l2*sin(phi2)*cos(theta2), l1*sin(phi1)*sin(theta1) + l2*sin(phi2)*sin(theta2) + 0.2*cos(t) - 0.2, -L + l1*cos(phi1) + l2*cos(phi2))

In [15]:
# get second order form
endefXdd = sm.diff(sm.diff(endefX,t),t)
endefYdd = sm.diff(sm.diff(endefY,t),t)
endefZdd = sm.diff(sm.diff(endefZ,t),t)
endefXdd, endefYdd, endefZdd

(-l1*sin(phi1)*sin(theta1)*theta1'' - l1*sin(phi1)*cos(theta1)*phi1'**2 - l1*sin(phi1)*cos(theta1)*theta1'**2 - 2*l1*sin(theta1)*cos(phi1)*phi1'*theta1' + l1*cos(phi1)*cos(theta1)*phi1'' - l2*sin(phi2)*sin(theta2)*theta2'' - l2*sin(phi2)*cos(theta2)*phi2'**2 - l2*sin(phi2)*cos(theta2)*theta2'**2 - 2*l2*sin(theta2)*cos(phi2)*phi2'*theta2' + l2*cos(phi2)*cos(theta2)*phi2'', -l1*sin(phi1)*sin(theta1)*phi1'**2 - l1*sin(phi1)*sin(theta1)*theta1'**2 + l1*sin(phi1)*cos(theta1)*theta1'' + l1*sin(theta1)*cos(phi1)*phi1'' + 2*l1*cos(phi1)*cos(theta1)*phi1'*theta1' - l2*sin(phi2)*sin(theta2)*phi2'**2 - l2*sin(phi2)*sin(theta2)*theta2'**2 + l2*sin(phi2)*cos(theta2)*theta2'' + l2*sin(theta2)*cos(phi2)*phi2'' + 2*l2*cos(phi2)*cos(theta2)*phi2'*theta2' - 0.2*cos(t), -l1*sin(phi1)*phi1'' - l1*cos(phi1)*phi1'**2 - l2*sin(phi2)*phi2'' - l2*cos(phi2)*phi2'**2)

In [16]:
A11 = endefXdd.coeff(theta1dd)
A21 = endefXdd.coeff(theta2dd)
A31 = endefXdd.coeff(phi1dd)
A41 = endefXdd.coeff(phi2dd)

A12 = endefYdd.coeff(theta1dd)
A22 = endefYdd.coeff(theta2dd)
A32 = endefYdd.coeff(phi1dd)
A42 = endefYdd.coeff(phi2dd)

A13 = endefZdd.coeff(theta1dd)
A23 = endefZdd.coeff(theta2dd)
A33 = endefZdd.coeff(phi1dd)
A43 = endefZdd.coeff(phi2dd)

A = sm.Matrix([[A11, A21, A31, A41], [A12, A22, A32, A42], [A13, A23, A33, A43]])
A

Matrix([
[-l1*sin(phi1)*sin(theta1), -l2*sin(phi2)*sin(theta2), l1*cos(phi1)*cos(theta1), l2*cos(phi2)*cos(theta2)],
[ l1*sin(phi1)*cos(theta1),  l2*sin(phi2)*cos(theta2), l1*sin(theta1)*cos(phi1), l2*sin(theta2)*cos(phi2)],
[                        0,                         0,            -l1*sin(phi1),            -l2*sin(phi2)]])

In [17]:
b1 = endefXdd - sm.expand(A11)*theta1dd - sm.expand(A21)*theta2dd - sm.expand(A31)*phi1dd - sm.expand(A41)*phi2dd 
b2 = endefYdd - sm.expand(A12)*theta1dd - sm.expand(A22)*theta2dd - sm.expand(A32)*phi1dd - sm.expand(A42)*phi2dd 
b3 = endefZdd - sm.expand(A13)*theta1dd - sm.expand(A23)*theta2dd - sm.expand(A33)*phi1dd - sm.expand(A43)*phi2dd 

b = -sm.Matrix([b1,b2,b3])
b

Matrix([
[             l1*sin(phi1)*cos(theta1)*phi1'**2 + l1*sin(phi1)*cos(theta1)*theta1'**2 + 2*l1*sin(theta1)*cos(phi1)*phi1'*theta1' + l2*sin(phi2)*cos(theta2)*phi2'**2 + l2*sin(phi2)*cos(theta2)*theta2'**2 + 2*l2*sin(theta2)*cos(phi2)*phi2'*theta2'],
[l1*sin(phi1)*sin(theta1)*phi1'**2 + l1*sin(phi1)*sin(theta1)*theta1'**2 - 2*l1*cos(phi1)*cos(theta1)*phi1'*theta1' + l2*sin(phi2)*sin(theta2)*phi2'**2 + l2*sin(phi2)*sin(theta2)*theta2'**2 - 2*l2*cos(phi2)*cos(theta2)*phi2'*theta2' + 0.2*cos(t)],
[                                                                                                                                                                                                       l1*cos(phi1)*phi1'**2 + l2*cos(phi2)*phi2'**2]])

In [18]:
# ode func for scipy solve_ivp
theta1d_func = sm.lambdify(theta1d, theta1d)
theta2d_func = sm.lambdify(theta2d, theta2d)
phi1d_func = sm.lambdify(phi1d, phi1d)
phi2d_func = sm.lambdify(phi2d, phi2d)

t_span = (0, 50)
g_ = 0
m1_ = 1
m2_ = 1
l1_ = 1
l2_ = 1
tq1_ = 0
tq2_ = 0
tq3_ = 0
tq4_ = 0
c1_ = 0.5
c2_ = 0.5
c3_ = 0.5
c4_ = 0.5
y0 = [30,0,-30,0,0,0,0]  # Initial state of the system


In [ ]:
h_sn = (A*(M**(-1/2))).subs({gty: g_,l1: l1_, l2: l2_, m1: m1_, m2: m2_, c1:c1_,c2:c2_,c3:c3_,c4:c4_,tau1:tq1_,tau2:tq2_,tau3:tq3_,tau4:tq4_})
j_sn = (M**(1/2)).subs({gty: g_,l1: l1_, l2: l2_, m1: m1_, m2: m2_, c1:c1_,c2:c2_,c3:c3_,c4:c4_,tau1:tq1_,tau2:tq2_,tau3:tq3_,tau4:tq4_})
k_sn = (b-(A*(M.inv()*F))).subs({gty: g_,l1: l1_, l2: l2_, m1: m1_, m2: m2_, c1:c1_,c2:c2_,c3:c3_,c4:c4_,tau1:tq1_,tau2:tq2_,tau3:tq3_,tau4:tq4_})
F_sn = F.subs({gty: g_,l1: l1_, l2: l2_, m1: m1_, m2: m2_, c1:c1_,c2:c2_,c3:c3_,c4:c4_,tau1:tq1_,tau2:tq2_,tau3:tq3_,tau4:tq4_})
M_sn = M.subs({gty: g_,l1: l1_, l2: l2_, m1: m1_, m2: m2_, c1:c1_,c2:c2_,c3:c3_,c4:c4_,tau1:tq1_,tau2:tq2_,tau3:tq3_,tau4:tq4_})

In [ ]:
def dSdt(t,S):
    thet1_, dtheta1_, thet2_, dtheta2_,phi1_,dphi1_,phi2_,dphi2_ = S
    
    print(t, S)
    
    h_n = h_sn.subs({Symbol('t'):t,theta1:thet1_ ,theta2:thet2_,theta1d:dtheta1_, theta2d:dtheta2_,
                    phi1:phi1_ ,phi2:phi2_,phi1d:dphi1_, phi2d:dphi2_}).evalf()
    j_n = j_sn.subs({Symbol('t'):t,theta1:thet1_ ,theta2:thet2_,theta1d:dtheta1_, theta2d:dtheta2_,
                    phi1:phi1_ ,phi2:phi2_,phi1d:dphi1_, phi2d:dphi2_}).evalf()
    k_n = k_sn.subs({Symbol('t'):t,theta1:thet1_ ,theta2:thet2_,theta1d:dtheta1_, theta2d:dtheta2_,
                    phi1:phi1_ ,phi2:phi2_,phi1d:dphi1_, phi2d:dphi2_}).evalf()
    psu_i = h_n.pinv()
    Fc = (j_n * psu_i * k_n)
    F_n = F_sn.subs({Symbol('t'):t,theta1:thet1_ ,theta2:thet2_,theta1d:dtheta1_, theta2d:dtheta2_,
                    phi1:phi1_ ,phi2:phi2_,phi1d:dphi1_, phi2d:dphi2_}).evalf()
    rf = F_n + Fc
    ra = M_sn.inv().subs({Symbol('t'):t,theta1:thet1_ ,theta2:thet2_,theta1d:dtheta1_, theta2d:dtheta2_,
                    phi1:phi1_ ,phi2:phi2_,phi1d:dphi1_, phi2d:dphi2_}).evalf() * rf
    return [
        theta1d_func(dtheta1_),
        ra[0],
        theta2d_func(dtheta2_),
        ra[1],
        phi1d_func(dphi1_),
        ra[2],
        phi2d_func(dphi2_),
        ra[3]
    ]


In [ ]:
solve_ivp_rk45 = solve_ivp(dSdt,t_span, y0, method="LSODA")